# Bird classification using deeplearning
The goal of this notebook is to classify birds images in function of their species.

Data are from:
https://darwin.v7labs.com/v7-open-datasets/bird-species

In [ ]:
#Libraries declaration
import json
import pandas as pd
import numpy as np
import urllib.request
import os
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split

Rename images from 0 to n (the number of images).

In [ ]:
x=0
for i in range(0, 10000):
    if(os.path.exists('data bird/'+str(i) +'.json')):
        os.rename('data bird/'+str(i) +'.json','data bird/'+str(x) +'.json')
        x+=1
    


Loading data in a dataframe

In [ ]:
df = pd.DataFrame()
data = []
for i in range(0, 1909):
  with open('data bird/'+str(i) +'.json', encoding='utf-8') as project_file: 
    df = pd.concat([df, pd.json_normalize(data)], ignore_index=True)
    data = json.load(project_file) 

df.head()


In [ ]:
anns = []
for i in range(0, 1908):
    pairs = df.annotations[i]
    string = json.dumps(pairs[0])
    print(json.loads(string)['name'])
    anns.append(json.loads(string)['name'])
df['ann']=anns


In [ ]:
df.head()

In [ ]:
nb_ann = 0
ann_name = []
i=0
for i in range(1, len(df)):
    if df.ann[i] not in ann_name:
        ann_name.append(df.ann[i])
        nb_ann
        print(df.ann[i], i)

Split dataframe between training and testing

In [ ]:
X_train,X_val=train_test_split(df,test_size=0.2)

print(pd.DataFrame(X_train).groupby(['ann']).size())

In [ ]:
X_train.head()

In [ ]:

X_train.reset_index(drop=True, inplace=True)
for i in range(0, len(X_train)):
  path = "train/" + X_train.ann[i]
  if not os.path.exists(path):
    os.mkdir(path)
  urllib.request.urlretrieve(X_train['image.url'][i],"train/"+X_train.ann[i]+"/image"+str(i)+'.jpg')
  print('Téléchargement réussi ' +str(i))


In [ ]:
X_val.reset_index(drop=True, inplace=True)
for i in range(0, len(X_val)):
  path = "test/" + X_val.ann[i]
  if not os.path.exists(path):
    os.mkdir(path)
  urllib.request.urlretrieve(X_val['image.url'][i],"test/"+X_val.ann[i]+"/image"+str(i)+'.jpg')
  print('Téléchargement réussi ' +str(i))

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split


from tensorflow.keras.optimizers import Adam 
from keras.metrics import categorical_crossentropy
from keras.models import load_model
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
train_path="train"
test_path="test"

In [ ]:
data_gen = dict(rescale=1./255, 
                rotation_range=35, 
                zoom_range = 0.3,
                shear_range=0.2,
                channel_shift_range=0.3,
                fill_mode='nearest',
                cval=0.3,
                width_shift_range=0.2,
                height_shift_range=0.2, 
                horizontal_flip = 'true',
                featurewise_std_normalization=True,
                samplewise_center=True,
                samplewise_std_normalization=True,
                zca_whitening=True,
                zca_epsilon=25,
                brightness_range=0.3,
                vertical_flip=True
                )

train_datagen = ImageDataGenerator(data_gen)

test_datagen = ImageDataGenerator(data_gen)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path, 
                                                    target_size=(224, 224), 
                                                    batch_size=128, 
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_path, 
                                                    target_size=(224, 224), 
                                                    batch_size=128, 
                                                    class_mode='categorical')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(Dropout(0.2))
model.add(Dense(16))
model.add(Dropout(0.2))
model.add(Dense(8))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [ ]:
ad = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=ad,
              metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
    model.fit(train_generator, 
                    shuffle=True,
                    epochs=17, 
                    validation_data=test_generator)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt


img = image.load_img('bird4.jfif', target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
prediction = model.predict_classes(img_batch)
print(ann_name[int(prediction)])
